# Exercise 8: k-Means Clustering and Neural Gas

**Note**: Please insert the names of all participating students:
1. 
2. 
3. 

## Preamble
The following code downloads and imports all necessary files and modules into the virtual machine of Colab. Please make sure to execute it before solving this exercise. This mandatory preamble will be found on all exercise sheets.

In [0]:
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/inb-uni-luebeck/cs4405.git'
    os.chdir('cs4405')

import numpy as np
from matplotlib import pyplot as plt
from utils import utils_8 as utils

## Exercise 8.1: k-Means Clustering

In many cases, the points in a dataset can be grouped into several clusters of points that lie close together. Each `cluster` can be described by a single representative point, the `centroid` or cluster center. In the first part of this exercise, we will study the k-Means Clustering algorithm, a simple algorithm that learns a representation of clusters.

We have $N$ `samples` $\boldsymbol{x}_{\mu} \in \mathbb{R}^{d}$, $\mu = 1,\dots,N$ and want to find $k$ `codebook_vectors` $\boldsymbol{w}_{i} \in \mathbb{R}^d$, $i = 1,\dots,k$ that represent the clusters in the data. Each data point $\boldsymbol{x}_{\mu}$ is assigned to the codebook vector that has the smallest euclidean distance to it. We call the index of this codebook vector $i^{∗}$:

$$
\begin{align}
    i^∗ := \underset{i}{\operatorname{argmin}} \lVert\boldsymbol{x}_{\mu} − \boldsymbol{w}_{i}\rVert_2.
\end{align}
$$

Here is the pseudocode for the k-Means Clustering algorithm: 

for $t=1$ to $t_{\max}$
> for $\mu=$ np.random.permutation$(N)$
>> - Find the closest codebook vector $\boldsymbol{w}_{i^{*}}$ to the data point $\boldsymbol{x}_{\mu}$. 
- Update the codebook vector $\boldsymbol{w}_{i^{*}}$ with $\boldsymbol{w}_{i^{*}} = \boldsymbol{w}_{i^{*}} + \varepsilon_{t} \left( \boldsymbol{x}_{\mu} − \boldsymbol{w}_{i^{*}} \right)$.

where $\varepsilon_t$ is the `learning_rate` that exponentially decays from an `initial_learning_rate` $\varepsilon_{start}$ to an `end_learning_rate` $\varepsilon_{end}$ as the number of performed learning `epochs` $t$ increases:

$$
\begin{align}
  \varepsilon_t = \varepsilon_{start}\left(\frac{\varepsilon_{end}}{\varepsilon_{start}}\right)^{t/t_{max}}.
\end{align}
$$

**Tasks**:
- Implement the k-Means Clustering algorithm in Python 
- Test your implementation several times varying the `codebook_size` $k$, the `initial_learning_rate` $\varepsilon_{start}$, and the `end_learning_rate` $\varepsilon_{end}$

**Programming Hints**:
 - Try to avoid for-loops when computing the distance of `sample` $\boldsymbol{x}_{\mu}$ to each `codebook_vector` $\boldsymbol{w}_{i}$ (`help(np.linalg.norm)`)

In [0]:
def k_means(samples, codebook_size, n_epochs, 
            initial_learning_rate, end_learning_rate):
  
  # n_samples: number of samples / n_features: number of features
  n_samples, n_features = samples.shape
    
  # TODO: randomly initialize codebook_vectors in range [-5, 5]
  codebook_vectors = 

  for epoch in range(n_epochs):
    
    # TODO: calculate the current learning rate
    learning_rate = 

    # generate randomly permuted index array
    indexes = np.random.permutation(n_samples)
        
    # iterate through all indexes in the index array
    for index in indexes:
      
      # get current sample
      sample = samples[index]

      # TODO: calculate the euclidean distances between sample and each codebook vector
      distances = 

      # TODO: get the index of the closest codebook vector to sample
      index_min = 

      # TODO: update the codebook vector
      codebook_vectors[index_min, :] = 

      yield {'codebook_vectors': codebook_vectors}


# load the dataset
samples = utils.load_data('data/data_8.npz')

# TODO: set the k-means parameters
codebook_size = 
n_epochs = 
initial_learning_rate = 
end_learning_rate = 

generator = k_means(samples, codebook_size, n_epochs, 
                    initial_learning_rate, end_learning_rate)
animation = utils.Animation(samples, codebook_size)
animation.animate(generator,
                  max_frames=100)

## Exercise 8.2: Neural Gas

The Neural Gas algorithm is very similar to the above k-Means Clustering algorithm, but for a given `sample` it adapts all`codebook_vectors` in a *soft-competitive* fashion, instead of *hard-competitively* changing only the winner $i^{∗}\left( \boldsymbol{x}_{\mu} \right)$. 

For codebook adaptation we use a neighborhood function $\lambda_t$ that determines how much close-by codebook vectors are attracted by the current `sample` $\boldsymbol{x}_{\mu}$. Just like $\varepsilon_t$, Neural Gas cools the `neighborhood_radius` $\lambda_t$ down from an `initial_neighborhood_radius` $\lambda_{start}$ to a `end_neighborhood_radius` $\lambda_{end}$:

$$
\begin{align}
  \lambda_t = \lambda_{start}\left(\frac{\lambda_{end}}{\lambda_{start}}\right)^{t/t_{max}}.
\end{align}
$$

All $k$ `codebook_vectors` $\boldsymbol{w}_{i}$ are updated as follows:

$$
\begin{align}
  \boldsymbol{w}_{i} = \boldsymbol{w}_{i} + \varepsilon_t e^{\frac{-r_{i} \left( \boldsymbol{x}_{\mu} \right)}{\lambda_{t}}}(\boldsymbol{x}_{\mu} − \boldsymbol{w}_{i}).
\end{align}
$$

where $r_{i} \left( \boldsymbol{x}_{\mu} \right)$ is the `rank` of `codebook_vector` $\boldsymbol{w}_{i}$:

$$
\begin{align}
  r_{i} \left( \boldsymbol{x}_{\mu} \right) = \lvert \left\{ j \mid  \lVert \boldsymbol{x}_{\mu} − \boldsymbol{w}_{j} \rVert_2 < \lVert \boldsymbol{x}_{\mu} − \boldsymbol{w}_{i} \rVert_2 \right\} \rvert,
\end{align}
$$ 

i.e. the number of codebook vectors $\boldsymbol{w}_{j}$ that have a smaller euclidean distance to $\boldsymbol{x}_{\mu}$ than $\boldsymbol{w}_{i}$.

**Tasks**:
- Implement the Neural Gas algorithm in Python 
- Test your implementation several times varying the `codebook_size` $k$, the `initial_neighborhood_radius` $\lambda_{start}$, and the `end_neighborhood_radius` $\lambda_{end}$

**Programming Hints**:
- Try to avoid for-loops when updating all codebook vectors. 
- Note that broadcasting only works for the last dimension of a tensor, therefore you might need to change its shape.

**Questions**:
- What differences do you notice between representations learned by k-Means Clustering and Neural Gas?

**Answers**:
- 


In [0]:
def neural_gas(samples, codebook_size, n_epochs, 
               initial_learning_rate, end_learning_rate, 
               initial_neighborhood_radius, end_neighborhood_radius):

  # n_samples: number of samples / n_features: number of features
  n_samples, n_features = samples.shape

  # TODO: randomly initialize codebook_vectors in range [-5, 5]
  codebook_vectors = 

  for epoch in range(n_epochs):
    
    # TODO: calculate the current learning rate
    learning_rate = 
        
    # TODO: calculate the neighborhood radius
    neighborhood_radius = 

    # generate randomly permuted index array
    indexes = np.random.permutation(n_samples)
        
    # iterate through all indexes in the index array
    for index in indexes:
      
      # get current sample
      sample = samples[index]

      # TODO: calculate the euclidean distances between sample and each codebook vector
      distances = 

      # TODO: compute the rank of each codebook vector
      ranks = 

      # TODO: update all codebook vectors
      codebook_vectors = 

      yield {'codebook_vectors': codebook_vectors}


# load the dataset
samples = utils.load_data('data/data_8.npz')

# TODO: set the neural gas parameters
codebook_size = 
n_epochs = 
initial_learning_rate = 
end_learning_rate = 
initial_neighborhood_radius = 
end_neighborhood_radius = 

generator = neural_gas(samples, codebook_size, n_epochs, 
                       initial_learning_rate, end_learning_rate, 
                       initial_neighborhood_radius, end_neighborhood_radius)
animation = utils.Animation(samples, codebook_size)
animation.animate(generator,
                  max_frames=100)